In [21]:
import tensorflow as tf
import sys
print(sys.version)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

2.7.9 (default, Jan  7 2015, 11:49:12) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.56)]


In [216]:
class MF_RS():
    def __init__(self, numUsers, numSongs, embedding_dim, reg_lambda=0.01, conf_lambda=1.0):
        
        #hyper parameters
        self.batch_size = 5000;
        self.numUsers = numUsers
        self.numSongs = numSongs
        self.epochs = 100
        self.reg_lambda = reg_lambda
        self.conf_lambda = conf_lambda
        
        #embedding matricies for users and songs
        self.userMat = tf.Variable(tf.random_normal([numUsers, embedding_dim]))
        self.songMat = tf.Variable(tf.random_normal([numSongs, embedding_dim]))
        self.userBias = tf.Variable(tf.random_normal([numUsers]))
        self.songBias = tf.Variable(tf.random_normal([numSongs]))
        self.overallBias = tf.Variable(tf.random_normal([1]))
        self.C_user = tf.Variable(.1*tf.ones([numUsers]))
        self.C_song = tf.Variable(.1*tf.ones([numSongs]))
        
        #input tensors for songs, usres, ratings
        self.users = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.songs = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.rating = tf.placeholder(tf.float32, shape = (self.batch_size))
        
        #map each user/song to its feature vector
        self.U = tf.nn.embedding_lookup(self.userMat, self.users)
        self.W = tf.nn.embedding_lookup(self.songMat, self.songs)
        # bias
        self.U_bias = tf.nn.embedding_lookup(self.userBias, self.users)
        self.W_bias = tf.nn.embedding_lookup(self.songBias, self.songs)
        # confidence params
        self.C_ui = tf.maximum(0.0, tf.nn.embedding_lookup(self.C_user, self.users))
        self.C_sj = tf.maximum(0.0, tf.nn.embedding_lookup(self.C_song, self.songs))

        
        #predicted rating is dot product of user and song
        bias = self.U_bias+self.W_bias+self.overallBias
        pq = tf.reduce_sum(tf.mul(self.U, self.W), 1)
        self.yhat = pq + bias
            
        # l2 reg
        self.confidence_reg = self.conf_lambda * tf.reduce_sum(tf.exp(-self.C_ui) + tf.exp(-self.C_sj))
        self.l2_reg = self.reg_lambda * ( tf.reduce_sum((tf.square(self.U) + tf.square(self.W))) + 
                                         tf.reduce_sum(tf.square(self.U_bias) + tf.square(self.W_bias)))
        self.reg = self.confidence_reg + self.l2_reg
        self.error = tf.reduce_mean(self.C_ui * self.C_sj * tf.nn.l2_loss(self.yhat - self.rating))
        self.cost = (self.error + self.reg)/1e7
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate = .01).minimize(self.cost)
        
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())    
        
    def train(self, users, songs, ratings):
        
        for i in range(self.epochs):
            
            avg_cost = 0
            perm = np.random.permutation(len(ratings))
            num_batches = len(ratings) // self.batch_size
            
            for b_idx in range(num_batches):
                
                batch = perm[self.batch_size * b_idx:self.batch_size * (b_idx + 1)]
                #if b_idx == num_batches - 1:
                #    batch = perm[self.batch_size * b_idx:]
                    
                users_batch = users[batch]
                songs_batch = songs[batch]
                ratings_batch = ratings[batch]
                                
                avg_cost += self.session.run([self.cost, self.optimizer],
                                  {self.users:users_batch, self.songs:songs_batch, self.rating:ratings_batch})[0]
                
            print(avg_cost/num_batches)

In [169]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
c = np.array([4, 3, 2, 5, 1])
#unique users / songs
uni_a = movieratings['userId'].unique()
uni_b = movieratings['movieId'].unique()

#dict mapping the id to an index
a_map = dict(zip(uni_a,range(len(uni_a))))
b_map = dict(zip(uni_b,range(len(uni_b))))

user_idx =  np.array([ a_map[user] for user in users])
song_idx =  np.array([ b_map[song] for song in songs])
model = MF_RS(len(uni_a), len(uni_b), 7)
print(model.session.run([model.cost, model.optimizer], {model.users:a, model.songs:b, model.rating:c}))


ValueError: Cannot feed value of shape (5,) for Tensor u'Placeholder_177:0', which has shape '(50000,)'

In [110]:
movieratings = pd.read_csv('ratings.csv')

In [111]:
print(len(movieratings['userId'].unique()))
print(len(movieratings['movieId'].unique()))

671
9066


In [112]:
movieratings.describe()
movieratings[['userId', 'movieId','rating']].as_matrix()

array([[  1.00000000e+00,   3.10000000e+01,   2.50000000e+00],
       [  1.00000000e+00,   1.02900000e+03,   3.00000000e+00],
       [  1.00000000e+00,   1.06100000e+03,   3.00000000e+00],
       ..., 
       [  6.71000000e+02,   6.36500000e+03,   4.00000000e+00],
       [  6.71000000e+02,   6.38500000e+03,   2.50000000e+00],
       [  6.71000000e+02,   6.56500000e+03,   3.50000000e+00]])

In [113]:
def getDfSummary(input_data):
    output_data = input_data.describe(include = 'all').T
    var = pd.DataFrame(data = {'nanvals': pd.Series(), 'number_distinct': pd.Series()})
    for i in range(len(input_data.columns)):
        nanvals = input_data.ix[:,i].isnull().sum()
        number_distinct = len(input_data.ix[:,i].value_counts())
        var = var.append(pd.DataFrame([[nanvals, number_distinct]], columns = ['nanvals', 'number_distinct']))
    var.index = output_data.index.values
    output_data['nanvals'] = var['nanvals']
    output_data['number_distinct'] = var['number_distinct']
    return output_data
output_data = getDfSummary(movieratings)

In [114]:
output_data

,count,mean,std,min,25%,50%,75%,max,nanvals,number_distinct
userId,100004,3.470113e+02,1.951638e+02,1.0,182,3.670000e+02,5.200000e+02,671,0,671
movieId,100004,1.254866e+04,2.636920e+04,1.0,1028,2.406500e+03,5.418000e+03,163949,0,9066
rating,100004,3.543608e+00,1.058064e+00,0.5,3,4.000000e+00,4.000000e+00,5,0,10
timestamp,100004,1.129639e+09,1.916858e+08,789652009.0,965847824,1.110422e+09,1.296192e+09,1476640644,0,78141


In [115]:
movieratings.keys()

Index([u'userId', u'movieId', u'rating', u'timestamp'], dtype='object')

In [116]:


users = movieratings.ix[:,0].values
songs = movieratings.ix[:,1].values
ratings = movieratings.ix[:,2].values

#unique users / songs
uni_users = movieratings['userId'].unique()
uni_songs = movieratings['movieId'].unique()

#dict mapping the id to an index
user_map = dict(zip(uni_users,range(len(uni_users))))
song_map = dict(zip(uni_songs,range(len(uni_songs))))

user_idx =  np.array([ user_map[user] for user in users])
song_idx =  np.array([ song_map[song] for song in songs])

print(len(uni_users),len(uni_songs))

(671, 9066)


In [217]:
songmodel = MF_RS(671, 9066, 11, reg_lambda=0.001, conf_lambda=1000)

In [218]:
np.random.seed(1)
songmodel.train(user_idx, song_idx, ratings)

0.904799446464
0.904567274451
0.904335406423
0.904103600979
0.903872275352
0.903641459346
0.903410935402
0.903180810809
0.902951067686
0.902721741796
0.902492779493
0.902264487743
0.902036440372
0.901808688045
0.901581260562
0.901354148984
0.901127520204
0.900901469588
0.900675854087
0.900450339913
0.900225445628
0.900000685453
0.899776285887
0.89955227077
0.899328675866
0.899105575681
0.898883479834
0.898661211133
0.898439443111
0.898217785358
0.897996735573
0.89777547121
0.897554817796
0.89733453989
0.897114849091
0.896895459294
0.896676686406
0.896458008885
0.896240073442
0.896022135019
0.895804214478
0.895587402582
0.895370662212
0.895153945684
0.894937738776
0.894721892476
0.894506508112
0.894291308522
0.894076818228
0.893862777948
0.893648511171
0.893435066938
0.893221834302
0.893008455634
0.892796248198
0.892583319545
0.892371305823
0.89215952754
0.891947823763
0.891736581922
0.891526374221
0.891315290332
0.891105368733
0.890896508098
0.890687155724
0.89047857821
0.890270200372


In [30]:
songmodel.songs.get_shape()

TensorShape([Dimension(32)])

In [61]:

# [ 2858  7190 57972  2105  2683] [ 4.5  4.   4.   3.5  3. ]
a = np.array([606 ,547, 468, 654 ,304])
b = np.array([ 2858,  7190, 57972,  2105 , 2683] )
c = np.array([ 4.5 , 4.,   4.,   3.5,  3. ])

songmodel.session.run([songmodel.cost, songmodel.optimizer],
                                  {songmodel.users:600*np.ones(32), songmodel.songs:np.ones(32), songmodel.rating:np.ones(32)})[0]

#model.session.run([model.yhat],
#                                  {model.users:a, model.songs:b, model.rating:c})[0]

2738.811

In [32]:
print(tf.all_variables())

[<tensorflow.python.ops.variables.Variable object at 0x10d621c10>, <tensorflow.python.ops.variables.Variable object at 0x111598790>, <tensorflow.python.ops.variables.Variable object at 0x111598d10>, <tensorflow.python.ops.variables.Variable object at 0x106c73e90>, <tensorflow.python.ops.variables.Variable object at 0x111652d90>, <tensorflow.python.ops.variables.Variable object at 0x106c9fad0>, <tensorflow.python.ops.variables.Variable object at 0x106c9f650>, <tensorflow.python.ops.variables.Variable object at 0x110fe1550>]


In [88]:
users

array([  1,   1,   1, ..., 671, 671, 671])

In [92]:
np.array([users,songs]).transpose().shape

(100004, 2)